In [115]:
import numpy as np
from scipy.ndimage.interpolation import shift
from collections import deque, namedtuple

In [116]:
def npshift(seq, n):
    return np.concatenate((seq[-n:], seq[:-n]))

In [117]:
bot_map = np.array(
      [[ 0,  1,  0,  0,  0, 0,  1,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 0,  1,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 0,  1,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 0,  1,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 0,  1,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 0,  1,  0,  0,  0]])

explorer_bot_map = np.array(
      [[ 0,  1,  0,  0,  0, 0,  1,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 0,  1,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 0,  1,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 0,  1,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 0,  1,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 0,  1,  0,  0,  0]])

In [118]:
neighbours = [(-1, -1), (-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0),
                      (1, -1), (0, -1)]
neighbours

[(-1, -1), (-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1)]

In [119]:
Location = namedtuple("Location", ("x", "y"))

In [120]:
tail = Location(6,6)
recruit_range = 5

y direction

In [121]:
for ny, nx in neighbours[5:6]: # ny = 1
#for ny, nx in neighbours[1:2]: # ny = -1

    x, y = tail.x + nx, tail.y + ny

    if ny:
        print(ny)
        start_point = y
        end_point =  np.clip(y + (ny * recruit_range) ,0, len(bot_map[0]))
        direction = ny        
        near_bots = bot_map[:,x][start_point : end_point : direction]
        print(near_bots)

1
[1 0 1]


In [122]:
_near_bots =(near_bots[:-1])
_near_bots_offset =(near_bots[1:])
print(_near_bots)
print(_near_bots_offset)

[1 0]
[0 1]


The indices of transition points between occupied and unoccupied cells:
Indices of first cells in transition pairs "_near_bots.index(i)"
Indices of second cells in transition pairs and hence the shift distance required "_near_bots.index(i)+1"

In [123]:
transition_points = np.argwhere(_near_bots!=_near_bots_offset) + 1
shift_distance = transition_points[0]
print(transition_points)
print(shift_distance)

[[1]
 [2]]
[1]


Construct a new map showing only explorer bots not in relay chains. 
For now we will use the map of all explorer bots. 
Check the index of the first trasnition point is occupied by a bot:

In [124]:
if explorer_bot_map[:,x][y + ny * shift_distance]:
    print(explorer_bot_map[:,x][y + ny * shift_distance])                        

In [125]:
    near_bots = explorer_bot_map[y, start_point : end_point : direction]
    print(near_bots)
    _near_bots = near_bots[:-1]
    _near_bots_offset = near_bots[1:]
    transition_points = np.argwhere(_near_bots!=_near_bots_offset) + 1
    transition_points

[0 0 0]


array([], shape=(0, 1), dtype=int64)

Check for a second transition back to unoccupied space within the sense range.
If not transition back to unoccupied space occurs, default to the last cell in the sense window:

In [126]:
    try:
        last_bot = transition_points[1]
    except IndexError:
        last_bot = len(near_bots)
    last_bot

3

In [127]:
last_bot += 1
last_bot

4

In [128]:
bot_map = np.array(
      [[ 0,  1,  0,  0,  0, 0,  1,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 0,  1,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 0,  1,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 0,  1,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 0,  1,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 0,  1,  0,  0,  0]])

In [129]:
    start_point = y
    end_point = np.clip(y + (last_bot * ny), 0, len(bot_map[0]))
    print(shift_distance)
    
    if ny == 1:
        print(ny)
        print(bot_map[:,x][start_point: end_point: direction]) 

        bot_map[:,x][start_point: end_point:direction] =\
            npshift(bot_map[:,x][start_point: end_point:direction], (shift_distance * -ny))
        
        print(bot_map[:,x][start_point: end_point]) 
                                                  
    elif ny == -1:  
        print(ny)
        print(bot_map[:,x][start_point : end_point : direction])
        
        bot_map[:,x][start_point: end_point:direction] =\
             npshift(bot_map[:,x][start_point: end_point:direction], (shift_distance * ny))
            
       
        print(bot_map[:,x][start_point : end_point : direction])



[1]
1
[1 0 1]
[0 1 1]


/home/hemma/.local/lib/python3.5/site-packages/ipykernel/__main__.py:2: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  from ipykernel import kernelapp as app


In [130]:
bot_map

array([[0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 0]])

In [131]:
seq = bot_map[:,x][start_point: end_point: direction]
n = 3
print(seq)

[0 1 1]


In [132]:
print(np.concatenate((seq[-n:], seq[:-n])))

[0 1 1]
